In [ ]:
import keras
import keras.applications
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from PIL import Image
import os
import gc

### Reading in Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
manual_label = pd.read_excel("/content/drive/MyDrive/Data403Materials/Data403Project3/Copy of Manual Labeling.xlsx")
manual_label.head()

,Image,Old,More than 1 person,Saturation (Strong/Weak),Nature,Blurry,Castle,Desert,Filter/Not,Europe,Straight,Shadows (Strong/Weak)
0,Project3\data403-project3\Training_Data\Alex\A...,1,0,1,1,1,0,0,0,0,1,0
1,Project3\data403-project3\Training_Data\Alex\A...,1,0,1,1,1,0,0,1,1,0,1
2,Project3\data403-project3\Training_Data\Alex\A...,1,0,0,0,1,0,0,0,0,0,0
3,Project3\data403-project3\Training_Data\Alex\A...,0,0,1,1,0,0,0,1,0,0,0
4,Project3\data403-project3\Training_Data\Alex\A...,0,0,1,1,0,0,0,0,0,1,1


In [ ]:
manual_label["Image"] = manual_label["Image"].str.split("\\").str[-1]

In [ ]:
alex = manual_label[manual_label["Image"].str.contains("Alex")]
alex["Target"] = 1
kelly = manual_label[manual_label["Image"].str.contains("Kelly")]
kelly["Target"] = 0

<ipython-input-5-095b37fda2ec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alex["Target"] = 1
<ipython-input-5-095b37fda2ec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kelly["Target"] = 0


In [ ]:
info_in_order = pd.concat([alex, kelly], axis = 0).iloc[:, 1:]
info_in_order

,Old,More than 1 person,Saturation (Strong/Weak),Nature,Blurry,Castle,Desert,Filter/Not,Europe,Straight,Shadows (Strong/Weak),Target
0,1,0,1,1,1,0,0,0,0,1,0,1
1,1,0,1,1,1,0,0,1,1,0,1,1
2,1,0,0,0,1,0,0,0,0,0,0,1
3,0,0,1,1,0,0,0,1,0,0,0,1
4,0,0,1,1,0,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
480,0,0,0,1,1,0,0,0,0,0,0,0
481,0,0,1,1,0,0,0,0,0,1,0,0
482,0,1,0,0,0,0,0,0,0,0,1,0
483,1,1,0,0,1,0,0,0,0,0,0,0


### Resnet

In [ ]:
set = keras.utils.image_dataset_from_directory(
    directory = "/content/drive/MyDrive/Data403Materials/Data403Project3/trainingData",
    image_size = (224, 224),
    shuffle = False,
    batch_size = 485
)

Found 485 files belonging to 2 classes.


In [ ]:
import os

# Check the files in the directory
directory = "/content/drive/MyDrive/Data403Materials/Data403Project3/trainingData"
files = os.listdir(directory)
files

['Alex', 'Kelly']

In [ ]:
import os

# Path to the training data directory
directory = "/content/drive/MyDrive/Data403Materials/Data403Project3/trainingData"

# List all subdirectories (which should correspond to the classes, e.g., 'Alex', 'Kelly')
subdirectories = os.listdir(directory)

# Initialize a dictionary to store the count of images for each class
class_image_counts = {}

# Iterate through each subdirectory and count the image files
for subdirectory in subdirectories:
    # Full path of the subdirectory
    subdir_path = os.path.join(directory, subdirectory)

    # Ensure it's a directory (not a file)
    if os.path.isdir(subdir_path):
        # Get all files in the subdirectory
        files_in_subdir = os.listdir(subdir_path)

        # Count only the image files (optional: filter by file extension)
        image_files = [f for f in files_in_subdir if f.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif'))]

        # Store the count of image files for this class
        class_image_counts[subdirectory] = len(image_files)

# Print the counts for each class
print(class_image_counts)


{'Alex': 256, 'Kelly': 229}


In [ ]:
accuracy_list = []
accuracy_list_with_manual = []
def cross_val(num_folds = 5):
    total_indices = np.array(list(range(0, 485)))
    np.random.shuffle(total_indices)
    for i in range(0, num_folds):
        # print(i * (total_indices.size-1) // num_folds, (i + 1) * (total_indices.size-1) // num_folds)
        test_indices = total_indices[i * (total_indices.size-1) // num_folds : (i + 1) * (total_indices.size-1) // num_folds]
        train_indices = total_indices[np.isin(total_indices, test_indices, invert = True)] # for some reason setdiff1d sorts in order... facepalm

        train_x = set.as_numpy_iterator().next()[0][train_indices]
        train_y = set.as_numpy_iterator().next()[1][train_indices]
        test_x = set.as_numpy_iterator().next()[0][test_indices]
        test_y = set.as_numpy_iterator().next()[1][test_indices]

        train = tf.data.Dataset.from_tensor_slices((tf.constant(train_x), tf.constant(train_y))).batch(32)
        test = tf.data.Dataset.from_tensor_slices((tf.constant(test_x), tf.constant(test_y))).batch(32)

        model = keras.applications.ResNet50(
            include_top = False,
            input_shape = (224, 224, 3)
        )
        model.trainable = False
        final_model = keras.models.Sequential()
        final_model.add(model)
        final_model.add(keras.layers.GlobalAveragePooling2D())
        final_model.add(keras.layers.Dense(64, activation = "relu"))
        final_model.add(keras.layers.Dropout(0.5))
        final_model.add(keras.layers.Dense(16, activation = "relu"))
        final_model.add(keras.layers.Dropout(0.3))
        final_model.add(keras.layers.Dense(1, activation = "sigmoid"))

        final_model.compile(optimizer = "adam",
                    loss = "binary_crossentropy",
                    metrics = ["accuracy"])
        final_model.fit(train, epochs = 10)

        accuracy_list.append(final_model.evaluate(test)[1])

        # combine both results and use log reg
        model_predictions = final_model.predict(train_x)
        train_image = info_in_order.iloc[train_indices]
        actual_y = train_image.iloc[:, -1]
        intermediate = train_image.iloc[:, :-1]
        actual_x = pd.concat([intermediate.reset_index(drop = True), pd.DataFrame(model_predictions, columns = ["prediction"])], axis = 1)

        log_reg = LogisticRegression()
        log_reg.fit(actual_x, actual_y)

        model_predictions = final_model.predict(test_x)
        test_image = info_in_order.iloc[test_indices]
        actual_y = test_image.iloc[:, -1]
        intermediate = test_image.iloc[:, :-1]
        actual_x = pd.concat([intermediate.reset_index(drop = True), pd.DataFrame(model_predictions, columns = ["prediction"])], axis = 1)
        accuracy_list_with_manual.append(accuracy_score(actual_y, log_reg.predict(actual_x)))
        print(accuracy_score(actual_y, log_reg.predict(actual_x)))




cross_val(num_folds = 5)
print(accuracy_list)

np.array(accuracy_list).mean()
np.array(accuracy_list_with_manual).mean()

set = keras.utils.image_dataset_from_directory(
    directory = "/content/drive/MyDrive/Data403Materials/Data403Project3/trainingData",
    image_size = (224, 224),
    shuffle = True,
    batch_size = 32
)
model = keras.applications.ResNet50(
    include_top = False,
    input_shape = (224, 224, 3)
)
model.trainable = False
final_model = keras.models.Sequential()
final_model.add(model)
final_model.add(keras.layers.GlobalAveragePooling2D())
final_model.add(keras.layers.Dense(64, activation = "relu"))
final_model.add(keras.layers.Dropout(0.5))
final_model.add(keras.layers.Dense(16, activation = "relu"))
final_model.add(keras.layers.Dropout(0.3))
final_model.add(keras.layers.Dense(1, activation = "sigmoid"))

final_model.compile(optimizer = "adam",
            loss = "binary_crossentropy",
            metrics = ["accuracy"])
final_model.fit(set, epochs = 10)
predictions = final_model.predict(set)


input_1 = pd.concat([info_in_order.iloc[:, :-1].reset_index(drop = True), pd.DataFrame(predictions, columns = ["prediction"])], axis = 1)

final_log_reg = LogisticRegression()
final_log_reg.fit(input_1, info_in_order.iloc[:, -1])

test_set_01 = manual_label[manual_label["Image"].str.contains("TestSetImage")]
test_set_01_images = keras.utils.image_dataset_from_directory(
    directory = "/content/drive/MyDrive/Data403Materials/Data403Project3/testData/TestSet01",
    image_size = (224, 224),
    labels = None,
    shuffle = False,
    batch_size = 24
)
model_predict_test = final_model.predict(test_set_01_images)
final_output = pd.concat([test_set_01.reset_index(drop = True), pd.DataFrame(model_predict_test, columns = ["prediction"])], axis = 1)
final_log_reg.predict(final_output.iloc[:, 1:])

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 524ms/step - accuracy: 0.5062 - loss: 0.9697
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - accuracy: 0.6520 - loss: 0.6793
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.6896 - loss: 0.5866
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.7697 - loss: 0.4231
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - accuracy: 0.7944 - loss: 0.4320
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.7739 - loss: 0.4271
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.8300 - loss: 0.3723
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - accuracy: 0.8328 - loss: 0.3718
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - accuracy: 0.8960 - loss: 0.2801
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.8756 - loss: 0.2937
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 98ms/step - accuracy: 0.8008 - loss: 0.3824
13/13 ━━━━━━━━━━━━━━━━━━

CNN

In [ ]:
np.array(accuracy_list).mean()

0.8099226713180542

In [ ]:
np.array(accuracy_list_with_manual).mean()

0.9566151202749141

### Performance

In [ ]:
set = keras.utils.image_dataset_from_directory(
    directory = "/content/drive/MyDrive/Data403Materials/Data403Project3/trainingData",
    image_size = (224, 224),
    shuffle = True,
    batch_size = 32
)
model = keras.applications.ResNet50(
    include_top = False,
    input_shape = (224, 224, 3)
)
model.trainable = False
final_model = keras.models.Sequential()
final_model.add(model)
final_model.add(keras.layers.GlobalAveragePooling2D())
final_model.add(keras.layers.Dense(64, activation = "relu"))
final_model.add(keras.layers.Dropout(0.5))
final_model.add(keras.layers.Dense(16, activation = "relu"))
final_model.add(keras.layers.Dropout(0.3))
final_model.add(keras.layers.Dense(1, activation = "sigmoid"))

final_model.compile(optimizer = "adam",
            loss = "binary_crossentropy",
            metrics = ["accuracy"])
final_model.fit(set, epochs = 10)
predictions = final_model.predict(set)


input_1 = pd.concat([info_in_order.iloc[:, :-1].reset_index(drop = True), pd.DataFrame(predictions, columns = ["prediction"])], axis = 1)

final_log_reg = LogisticRegression()
final_log_reg.fit(input_1, info_in_order.iloc[:, -1])

Found 485 files belonging to 2 classes.
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 18s 428ms/step - accuracy: 0.5345 - loss: 0.8271
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 176ms/step - accuracy: 0.6247 - loss: 0.6601
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 181ms/step - accuracy: 0.7276 - loss: 0.5180
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 181ms/step - accuracy: 0.7558 - loss: 0.4662
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 224ms/step - accuracy: 0.7755 - loss: 0.4602
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 184ms/step - accuracy: 0.7904 - loss: 0.4018
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 219ms/step - accuracy: 0.8105 - loss: 0.3759
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 187ms/step - accuracy: 0.8516 - loss: 0.3117
Epoch 9/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 186ms/step - accuracy: 0.8349 - loss: 0.3218
Epoch 10/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 184ms/step - accuracy: 0.8809 - loss: 0.2723
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 332ms/step


LogisticRegression()

In [ ]:
test_set_01 = manual_label[manual_label["Image"].str.contains("TestSetImage")]
test_set_01_images = keras.utils.image_dataset_from_directory(
    directory = "/content/drive/MyDrive/Data403Materials/Data403Project3/testData/TestSet01",
    image_size = (224, 224),
    labels = None,
    shuffle = False,
    batch_size = 24
)
model_predict_test = final_model.predict(test_set_01_images)
final_output = pd.concat([test_set_01.reset_index(drop = True), pd.DataFrame(model_predict_test, columns = ["prediction"])], axis = 1)
final_log_reg.predict(final_output.iloc[:, 1:])

Found 20 files.
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step


array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0])

In [ ]:
np.where(final_model.predict(test_set_01_images) > 0.5, 1, 0).reshape(-1, 1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step


array([[0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0]])

In [ ]:
test_set_02 = manual_label[manual_label["Image"].str.contains("TestSet02")]
test_set_02_images = keras.utils.image_dataset_from_directory(
    directory = "/content/drive/MyDrive/Data403Materials/Data403Project3/testData/TestSet02",
    image_size = (224, 224),
    labels = None,
    shuffle = False,
    batch_size = 24
)
model_predict_test = final_model.predict(test_set_02_images)
final_output = pd.concat([test_set_02.reset_index(drop = True), pd.DataFrame(model_predict_test, columns = ["prediction"])], axis = 1)
final_log_reg.predict_proba(final_output.iloc[:, 1:])

Found 24 files.
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step


array([[0.96620829, 0.03379171],
       [0.38245092, 0.61754908],
       [0.03626881, 0.96373119],
       [0.97065997, 0.02934003],
       [0.13689565, 0.86310435],
       [0.96901933, 0.03098067],
       [0.83305645, 0.16694355],
       [0.83700131, 0.16299869],
       [0.58728945, 0.41271055],
       [0.8361609 , 0.1638391 ],
       [0.96912091, 0.03087909],
       [0.98287134, 0.01712866],
       [0.96939904, 0.03060096],
       [0.97088924, 0.02911076],
       [0.96951077, 0.03048923],
       [0.94717635, 0.05282365],
       [0.97113409, 0.02886591],
       [0.36218169, 0.63781831],
       [0.97059287, 0.02940713],
       [0.37235845, 0.62764155],
       [0.96977775, 0.03022225],
       [0.91106782, 0.08893218],
       [0.98962042, 0.01037958],
       [0.83470594, 0.16529406]])

VGG16 (Type of CNN Model)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

# Load the image dataset from directory (ensure the folder structure is as explained)
image_data = tf.keras.utils.image_dataset_from_directory(
    directory="/content/drive/MyDrive/Data403Materials/Data403Project3/trainingData",
    image_size=(224, 224),  # Resize to 224x224
    shuffle=True,  # Shuffle the data
    batch_size=32  # Set batch size
)




Found 485 files belonging to 2 classes.


In [ ]:
# Load the VGG16 model with pre-trained weights, excluding the top (classification) layers
vgg_model = keras.applications.VGG16(
    include_top=False,
    input_shape=(224, 224, 3)
)
vgg_model.trainable = False  # Freeze the layers in VGG16 (we only use it as a feature extractor)

# Create a custom model using VGG16 as a base
final_model_vgg = keras.Sequential()
final_model_vgg.add(vgg_model)  # Add the VGG16 model
final_model_vgg.add(keras.layers.GlobalAveragePooling2D())  # Pool the features
final_model_vgg.add(keras.layers.Dense(64, activation="relu"))  # Add custom dense layers
final_model_vgg.add(keras.layers.Dropout(0.5))  # Dropout for regularization
final_model_vgg.add(keras.layers.Dense(16, activation="relu"))
final_model_vgg.add(keras.layers.Dropout(0.3))  # Dropout for regularization
final_model_vgg.add(keras.layers.Dense(1, activation="sigmoid"))  # Output layer for binary classification

# Compile the model
final_model_vgg.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Train the model on the image dataset
final_model_vgg.fit(image_data, epochs=10)




Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 12s 551ms/step - accuracy: 0.5127 - loss: 2.3986
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 14s 298ms/step - accuracy: 0.6040 - loss: 0.9053
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 197ms/step - accuracy: 0.5727 - loss: 0.8159
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 199ms/step - accuracy: 0.7060 - loss: 0.6647
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 254ms/step - accuracy: 0.6921 - loss: 0.6532
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 196ms/step - accuracy: 0.6862 - loss: 0.5985
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 197ms/step - accuracy: 0.6930 - loss: 0.5841
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 255ms/step - accuracy: 0.7113 - loss: 0.5756
Epoch 9/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 234ms/step - accuracy: 0.7027 - loss: 0.5550
Epoch 10/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 196ms/step - accuracy: 0.7567 - loss: 0.4734


In [ ]:
# Predict using the trained model
image_predictions_vgg = final_model_vgg.predict(image_data)

# Now, load the manual features (assuming you have the 'info_in_order' DataFrame)
# Merge the image predictions with the manual features (info_in_order should already have the manual features)
input_data_vgg = pd.concat([info_in_order.iloc[:, :-1].reset_index(drop=True), pd.DataFrame(image_predictions_vgg, columns=["prediction"])], axis=1)

# Train a Logistic Regression model using both the image features and manual features
log_reg_model_vgg = LogisticRegression()
log_reg_model_vgg.fit(input_data_vgg, info_in_order.iloc[:, -1])  # info_in_order.iloc[:, -1] is the target (labels)

# Predict with Logistic Regression on the combined data
log_reg_predictions_vgg = log_reg_model_vgg.predict(input_data_vgg)

# Evaluate the Logistic Regression model (you can use accuracy score or any other metric)
accuracy_vgg = accuracy_score(info_in_order.iloc[:, -1], log_reg_predictions_vgg)
print(f"Logistic Regression Accuracy using VGG16: {accuracy_vgg}")


16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 215ms/step
Logistic Regression Accuracy using VGG16: 0.9587628865979382


MobileNetV2 Model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

# Load the image dataset from directory (ensure the folder structure is as explained)
image_data = tf.keras.utils.image_dataset_from_directory(
    directory="/content/drive/MyDrive/Data403Materials/Data403Project3/trainingData",
    image_size=(224, 224),  # Resize to 224x224
    shuffle=True,  # Shuffle the data
    batch_size=32  # Set batch size
)

set = keras.utils.image_dataset_from_directory(
    directory = "/content/drive/MyDrive/Data403Materials/Data403Project3/trainingData",
    image_size = (224, 224),
    shuffle = False,
    batch_size = 485
)

Found 485 files belonging to 2 classes.
Found 485 files belonging to 2 classes.


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd

# Initialize accuracy lists
accuracy_list = []
accuracy_list_with_manual = []

def cross_val(num_folds=5):
    total_indices = np.array(list(range(0, 485)))
    np.random.shuffle(total_indices)
    for i in range(0, num_folds):
        test_indices = total_indices[i * (total_indices.size - 1) // num_folds : (i + 1) * (total_indices.size - 1) // num_folds]
        train_indices = total_indices[np.isin(total_indices, test_indices, invert=True)]

        train_x = set.as_numpy_iterator().next()[0][train_indices]
        train_y = set.as_numpy_iterator().next()[1][train_indices]
        test_x = set.as_numpy_iterator().next()[0][test_indices]
        test_y = set.as_numpy_iterator().next()[1][test_indices]

        train = tf.data.Dataset.from_tensor_slices((tf.constant(train_x), tf.constant(train_y))).batch(32)
        test = tf.data.Dataset.from_tensor_slices((tf.constant(test_x), tf.constant(test_y))).batch(32)

        # Use MobileNetV2 instead of ResNet50
        base_model = keras.applications.MobileNetV2(
            include_top=False,
            input_shape=(224, 224, 3)
        )
        base_model.trainable = False

        # Build the final model
        final_model = keras.models.Sequential()
        final_model.add(base_model)
        final_model.add(keras.layers.GlobalAveragePooling2D())
        final_model.add(keras.layers.Dense(64, activation="relu"))
        final_model.add(keras.layers.Dropout(0.5))
        final_model.add(keras.layers.Dense(16, activation="relu"))
        final_model.add(keras.layers.Dropout(0.3))
        final_model.add(keras.layers.Dense(1, activation="sigmoid"))

        final_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
        final_model.fit(train, epochs=10)

        # Evaluate the model
        accuracy_list.append(final_model.evaluate(test)[1])

        # Combine predictions with manual features for logistic regression
        model_predictions = final_model.predict(train_x)
        train_image = info_in_order.iloc[train_indices]
        actual_y = train_image.iloc[:, -1]
        intermediate = train_image.iloc[:, :-1]
        actual_x = pd.concat([intermediate.reset_index(drop=True), pd.DataFrame(model_predictions, columns=["prediction"])], axis=1)

        log_reg = LogisticRegression()
        log_reg.fit(actual_x, actual_y)

        model_predictions = final_model.predict(test_x)
        test_image = info_in_order.iloc[test_indices]
        actual_y = test_image.iloc[:, -1]
        intermediate = test_image.iloc[:, :-1]
        actual_x = pd.concat([intermediate.reset_index(drop=True), pd.DataFrame(model_predictions, columns=["prediction"])], axis=1)
        accuracy_list_with_manual.append(accuracy_score(actual_y, log_reg.predict(actual_x)))
        print(accuracy_score(actual_y, log_reg.predict(actual_x)))

# Run cross-validation
cross_val(num_folds=5)
print(accuracy_list)
print("Mean Accuracy (Base Model):", np.array(accuracy_list).mean())
print("Mean Accuracy (Combined Model):", np.array(accuracy_list_with_manual).mean())



9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 351ms/step - accuracy: 0.5372 - loss: 0.8655
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step - accuracy: 0.5323 - loss: 0.7266
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5837 - loss: 0.6932
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.5094 - loss: 0.7195
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5480 - loss: 0.6818
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.5483 - loss: 0.6855
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.6360 - loss: 0.6429
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.6327 - loss: 0.6516
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.6785 - loss: 0.6018
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.6313 - loss: 0.6367
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - accuracy: 0.6797 - loss: 0.6133
13/13 ━━━━━━━━━━━━━━━━━━━

In [ ]:
# Training the final model on the entire dataset
set = keras.utils.image_dataset_from_directory(
    directory="/content/drive/MyDrive/Data403Materials/Data403Project3/trainingData",
    image_size=(224, 224),
    shuffle=True,
    batch_size=32
)

base_model = keras.applications.MobileNetV2(
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = False

final_model = keras.models.Sequential()
final_model.add(base_model)
final_model.add(keras.layers.GlobalAveragePooling2D())
final_model.add(keras.layers.Dense(64, activation="relu"))
final_model.add(keras.layers.Dropout(0.5))
final_model.add(keras.layers.Dense(16, activation="relu"))
final_model.add(keras.layers.Dropout(0.3))
final_model.add(keras.layers.Dense(1, activation="sigmoid"))

final_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
final_model.fit(set, epochs=10)

# Predict using the final model
predictions = final_model.predict(set)

input_1 = pd.concat([info_in_order.iloc[:, :-1].reset_index(drop=True), pd.DataFrame(predictions, columns=["prediction"])], axis=1)
final_log_reg = LogisticRegression()
final_log_reg.fit(input_1, info_in_order.iloc[:, -1])

# Predict on TestSet01
test_set_01 = manual_label[manual_label["Image"].str.contains("TestSetImage")]
test_set_01_images = keras.utils.image_dataset_from_directory(
    directory="/content/drive/MyDrive/Data403Materials/Data403Project3/testData/TestSet01",
    image_size=(224, 224),
    labels=None,
    shuffle=False,
    batch_size=24
)
model_predict_test = final_model.predict(test_set_01_images)
final_output = pd.concat([test_set_01.reset_index(drop=True), pd.DataFrame(model_predict_test, columns=["prediction"])], axis=1)
final_predictions = final_log_reg.predict(final_output.iloc[:, 1:])
print("Final TestSet01 Predictions:", final_predictions)


Found 485 files belonging to 2 classes.
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 14s 296ms/step - accuracy: 0.4739 - loss: 0.9053
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 269ms/step - accuracy: 0.5073 - loss: 0.7186
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 175ms/step - accuracy: 0.5428 - loss: 0.6874
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 185ms/step - accuracy: 0.5767 - loss: 0.6728
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 234ms/step - accuracy: 0.5411 - loss: 0.6913
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 177ms/step - accuracy: 0.5520 - loss: 0.6882
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 177ms/step - accuracy: 0.6507 - loss: 0.6339
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 218ms/step - accuracy: 0.6226 - loss: 0.6414
Epoch 9/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 181ms/step - accuracy: 0.6550 - loss: 0.6238
Epoch 10/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 179ms/step - accuracy: 0.6877 - loss: 0.6079
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 351ms/step
Found 20 files.
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
Final

In [ ]:
np.where(final_model.predict(test_set_01_images) > 0.5, 1, 0).reshape(-1, 1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step


array([[0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1]])

In [ ]:
test_set_02 = manual_label[manual_label["Image"].str.contains("TestSet02")]
test_set_02_images = keras.utils.image_dataset_from_directory(
    directory = "/content/drive/MyDrive/Data403Materials/Data403Project3/testData/TestSet02",
    image_size = (224, 224),
    labels = None,
    shuffle = False,
    batch_size = 24
)
model_predict_test = final_model.predict(test_set_02_images)
final_output = pd.concat([test_set_02.reset_index(drop = True), pd.DataFrame(model_predict_test, columns = ["prediction"])], axis = 1)
final_log_reg.predict_proba(final_output.iloc[:, 1:])

Found 24 files.
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


array([[0.96708723, 0.03291277],
       [0.39172731, 0.60827269],
       [0.0347275 , 0.9652725 ],
       [0.97048127, 0.02951873],
       [0.13435028, 0.86564972],
       [0.96941146, 0.03058854],
       [0.83329848, 0.16670152],
       [0.83039235, 0.16960765],
       [0.58594653, 0.41405347],
       [0.82374975, 0.17625025],
       [0.96888419, 0.03111581],
       [0.98279362, 0.01720638],
       [0.96858121, 0.03141879],
       [0.9705966 , 0.0294034 ],
       [0.9680986 , 0.0319014 ],
       [0.94711149, 0.05288851],
       [0.97066519, 0.02933481],
       [0.37318534, 0.62681466],
       [0.97035043, 0.02964957],
       [0.36812186, 0.63187814],
       [0.96895808, 0.03104192],
       [0.91142666, 0.08857334],
       [0.98923148, 0.01076852],
       [0.83432813, 0.16567187]])